# Token Streaming: 토큰 단위 스트리밍

이 노트북에서는 **LLM 출력을 토큰 단위로 실시간 스트리밍**하는 방법을 배웁니다.

## 토큰 스트리밍이란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    토큰 스트리밍의 개념                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   노드 단위 스트리밍:                                              │
│   노드 완료 → 전체 응답 반환                                       │
│   "미국의 30대 대통령은 캘빈 쿨리지입니다..." (전체)              │
│                                                                    │
│   토큰 단위 스트리밍:                                              │
│   "미국" → "의" → "30" → "대" → "대통령" → "은" → ...          │
│   (실시간으로 글자가 나타남)                                       │
│                                                                    │
│   장점:                                                            │
│   • ChatGPT처럼 실시간 타이핑 효과                                 │
│   • 사용자가 기다리는 느낌 감소                                    │
│   • 긴 응답도 즉시 시작                                            │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 에이전트 그래프 준비

In [ ]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)


class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state['messages'])
    return {'messages': res}


builder = StateGraph(State)
builder.add_node('model', model_node)
builder.add_node('tools', ToolNode(tools))
builder.add_edge(START, 'model')
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')

# checkpointer 필요 (astream_events 사용 시)
graph = builder.compile(checkpointer=MemorySaver())

print("✅ 에이전트 그래프 준비 완료")

# 3. 토큰 스트리밍 (astream_events)

비동기 방식으로 토큰 단위 스트리밍을 구현합니다.

In [ ]:
import asyncio

async def stream_tokens():
    """
    astream_events를 사용한 토큰 스트리밍
    
    event['event'] == 'on_chat_model_stream' 일 때
    토큰 단위로 content를 받을 수 있음
    """
    input_data = {
        'messages': [
            HumanMessage(
                '미국 제30대 대통령의 사망 당시 나이는 몇 살이었나요?'
            )
        ]
    }
    config = {'configurable': {'thread_id': '1'}}
    
    print("=== 토큰 스트리밍 ===")
    print(f"질문: {input_data['messages'][0].content}\n")
    print("응답: ", end='', flush=True)
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        # LLM 스트리밍 이벤트 캡처
        if event["event"] == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end='', flush=True)  # 토큰 단위 출력
    
    print("\n\n✅ 스트리밍 완료")

# 비동기 실행
await stream_tokens()

# 4. 이벤트 유형 확인

In [ ]:
async def show_events():
    """
    다양한 이벤트 유형 확인
    """
    input_data = {
        'messages': [
            HumanMessage('안녕하세요, 간단히 인사해주세요.')
        ]
    }
    config = {'configurable': {'thread_id': '2'}}
    
    print("=== 이벤트 유형 ===")
    
    event_types = set()
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        event_type = event["event"]
        if event_type not in event_types:
            event_types.add(event_type)
            print(f"새 이벤트 유형: {event_type}")
    
    print(f"\n발견된 이벤트 유형: {len(event_types)}개")

await show_events()

# 5. 주요 이벤트 필터링

In [ ]:
async def filtered_stream():
    """
    주요 이벤트만 필터링하여 표시
    """
    input_data = {
        'messages': [
            HumanMessage('파이썬이란 무엇인가요? 한 문장으로 설명해주세요.')
        ]
    }
    config = {'configurable': {'thread_id': '3'}}
    
    print("=== 필터링된 이벤트 스트림 ===")
    
    async for event in graph.astream_events(input_data, config, version="v2"):
        event_type = event["event"]
        
        if event_type == "on_chain_start":
            name = event.get("name", "unknown")
            print(f"🚀 체인 시작: {name}")
            
        elif event_type == "on_chat_model_start":
            print(f"🤖 LLM 호출 시작")
            
        elif event_type == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(f"📝 토큰: '{content}'")
                
        elif event_type == "on_chat_model_end":
            print(f"✅ LLM 호출 완료")
            
        elif event_type == "on_chain_end":
            name = event.get("name", "unknown")
            if name == "LangGraph":
                print(f"🏁 그래프 완료")

await filtered_stream()

---

## 정리: Token Streaming

### 핵심 코드

```python
import asyncio

async def stream_tokens():
    async for event in graph.astream_events(input, config, version="v2"):
        if event["event"] == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end='', flush=True)

# 실행
await stream_tokens()  # 또는 asyncio.run(stream_tokens())
```

### 주요 이벤트 유형

| 이벤트 | 설명 | 용도 |
|--------|------|------|
| **on_chat_model_stream** | LLM 토큰 출력 | 실시간 응답 표시 |
| **on_chat_model_start** | LLM 호출 시작 | 로딩 표시 |
| **on_chat_model_end** | LLM 호출 완료 | 로딩 종료 |
| **on_chain_start/end** | 체인 시작/완료 | 진행 상황 추적 |
| **on_tool_start/end** | 도구 시작/완료 | 도구 실행 추적 |

### 주의 사항

- `astream_events`는 비동기 함수 (`async for` 필요)
- `checkpointer`가 필요함 (MemorySaver 등)
- `version="v2"` 권장

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**Interrupt**를 사용하여 그래프 실행을 중단하고 재개하는 방법을 배웁니다. (05-06번 노트북)